# Movie Recommender System

Jing Huang, Xiao Zhang,

The goal of this project is to predict the ratings that a user would give to movies not rated by the user, and find the movies most likely to be liked by user. Four different algorithms are implemented in this project:
* Simple Recommender Systems
    * Simple Recommender Systems based on Ratings
    * Popularity Based Recommender Systems
* Collabortive Filtering
    * SVD
    * User-based Collabortive Filtering

In [1]:
# import dash
# import dash_bootstrap_components as dbc
# import dash_html_components as html
# import dash_core_components as dcc
import numpy as np
import pandas as pd
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [2]:
ratings = pd.read_csv('ratings.dat', sep=':', header=None)
ratings = ratings.dropna(axis=1)
ratings.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']

movies = pd.read_csv('movies.dat', sep='::', header=None, encoding='latin-1',engine='python')
movies.columns = ['MovieID', 'Title', 'Genres']

users = pd.read_csv('users.dat', sep=':', header=None, encoding='latin-1')
users = users.dropna(axis=1)
users.columns = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### Number of Movies

In [3]:
# ratings.head()
#total number of movies
len(ratings['MovieID'].unique())

3706

### Number of unique users:

In [4]:
len(ratings['UserID'].unique())

6040

In [5]:
display(users.isna().sum().sort_values())

UserID        0
Gender        0
Age           0
Occupation    0
Zip-code      0
dtype: int64

### Get a list of genres in this dataset

In [6]:
genres = movies['Genres'].str.split('|')
genres_list = list(set(np.concatenate(genres).flat))
print(genres_list)


['Horror', "Children's", 'Drama', 'Sci-Fi', 'Comedy', 'Mystery', 'Western', 'Crime', 'Documentary', 'War', 'Thriller', 'Musical', 'Adventure', 'Film-Noir', 'Animation', 'Romance', 'Fantasy', 'Action']


In [ ]:
# sample= movies[movies['Title'].str.contains('Army of Darkness')]
# sample.head() 

# Simple Recommonder Systems
 ## recommondation based on ratings 

This simple recommender system does not require users information, it provides the same recommendations for all users based on the genre chosen by the user. For this simple recommender system, a genre is first selected by the user, then up to 6 movies are returned in the given genre.

In this algorithm, recommondation is provided based on averaged user ratings. We noticed that there are some movies that received only one 5-star rating, their averaged rating is thus 5. However, these moives should be excluded. Therefore, the movie data are first filtered, only the movies that received more ratings than at least 80% of the movies are considered. The procedure for this algorithm is as follows:
1.	Left join movie data with ratings data on the index ‘MovieID’
2.	Filter the movies data based on genre selected by the user
3.	Calculate number of ratings that each movie received, calculate 80% quantile as cutoff
4.	Filter out the movies that received less ratings than 80% of all other movies.
5.	Sort movies by averaged rating, select the top 5 movies as output.


In [18]:

def recc_genre_by_rating(movies,ratings,genre):
    movies = movies.copy()
    ratings = ratings.copy()
    mov_rating=movies.set_index('MovieID').join(ratings.set_index('MovieID'),how ='left',on ='MovieID')
    mov_rating =mov_rating[mov_rating['Genres'].str.contains(genre)]
    mov_rating['count']=mov_rating['UserID'].groupby(mov_rating['Title']).transform('count')
    #only movies that received more number of ratings than 90% of all the movies in the genre are considered
    cutoff = mov_rating['count'].quantile(0.8)
    mov_rating = mov_rating[mov_rating['count']>=cutoff].drop(['UserID','Timestamp'],axis=1)
    mov_rating_mean = mov_rating.groupby('Title').mean().sort_values(by=['Rating','count'], ascending=False)
    
    return mov_rating_mean.head(5)
    
#testing: only show results for the first two genres
for i in genres_list[:2]:
    
    recc_genre = recc_genre_by_rating(movies,ratings,i)
    print('reccomendations for the genre:',i,'\n')
    print(recc_genre.head())
    print('\n')

reccomendations for the genre: Romance 

                              Rating  count
Title                                      
Casablanca (1942)           4.412822   1669
Princess Bride, The (1987)  4.303710   2318
Graduate, The (1967)        4.245837   1261
Annie Hall (1977)           4.141679   1334
Shakespeare in Love (1998)  4.127480   2369


reccomendations for the genre: Adventure 

                                                      Rating  count
Title                                                              
Raiders of the Lost Ark (1981)                      4.477725   2514
Star Wars: Episode IV - A New Hope (1977)           4.453694   2991
Princess Bride, The (1987)                          4.303710   2318
Star Wars: Episode V - The Empire Strikes Back ...  4.292977   2990
Stand by Me (1986)                                  4.096919   1785




In [14]:
def recc_genre_by_rating_ID(movies,ratings,genre):
    movies = movies.copy()
    ratings = ratings.copy()
    mov_rating=movies.set_index('MovieID').join(ratings.set_index('MovieID'),how ='left',on ='MovieID')
    mov_rating =mov_rating[mov_rating['Genres'].str.contains(genre)]
    mov_rating['count']=mov_rating['UserID'].groupby(mov_rating['Title']).transform('count')
    #only movies that received more number of ratings than 90% of all the movies in the genre are considered
    cutoff = mov_rating['count'].quantile(0.8)
    mov_rating = mov_rating[mov_rating['count']>=cutoff].drop(['UserID','Timestamp'],axis=1)
    mov_rating_mean = mov_rating.groupby('MovieID').mean().sort_values(by=['Rating','count'], ascending=False)
    
    return mov_rating_mean.head(5)
    
#testing: only show results for the first two genres
for i in genres_list[:2]:
    
    recc_genre = recc_genre_by_rating_ID(movies,ratings,i)
    print('reccomendations for the genre:',i,'\n')
    print(recc_genre.head())
    print('\n')

reccomendations for the genre: Drama 

           Rating  count
MovieID                 
318      4.554558   2227
858      4.524966   2223
527      4.510417   2304
912      4.412822   1669
1193     4.390725   1725


reccomendations for the genre: Children's 

           Rating  count
MovieID                 
919      4.247963   1718
3114     4.218927   1585
1        4.146846   2077
1097     3.965183   2269
34       3.891491   1751


reccomendations for the genre: Animation 

           Rating  count
MovieID                 
3114     4.218927   1585
1        4.146846   2077
3751     3.879609   1329
2355     3.854375   1703
588      3.788305   1351


reccomendations for the genre: Crime 

           Rating  count
MovieID                 
858      4.524966   2223
50       4.517106   1783
1221     4.357565   1692
296      4.278213   2171
1213     4.275196   1657


reccomendations for the genre: Comedy 

           Rating  count
MovieID                 
1136     4.335210   1599
2858     4.3

## Recommendation based on popularity
In this approach, the popularity is measured based on number of user ratings. i.e.a movie receives more ratings is considered to be more popular. The general procedure is as follows:
1.	Left join movie data with ratings data on the index ‘MovieID’
2.	Filter the movies data based on genre selected by the user
3.	Calculate the averaged rating of each movie received, calculate 80% quantile as cutoff
4.	Filter out the movies that received less score than 80% of all other movies.
5.	Sort movies by number of ratings, select the top 5 movies as output.


In [9]:
def recc_genre_by_popularity(movies,ratings,genre):
    movies = movies.copy()
    ratings = ratings.copy()
    mov_rating=movies.set_index('MovieID').join(ratings.set_index('MovieID'),how ='left',on ='MovieID')
    mov_rating =mov_rating[mov_rating['Genres'].str.contains(genre)]
    mov_rating['count']=mov_rating['UserID'].groupby(mov_rating['Title']).transform('count')
    mov_rating['avg_rating']=mov_rating['Rating'].groupby(mov_rating['Title']).transform('mean')
    #only movies that received higher ratings than 90% of all the movies in the genre are considered
    cutoff = mov_rating['avg_rating'].quantile(0.8)
    mov_rating = mov_rating[mov_rating['avg_rating']>=cutoff].drop(['UserID','Timestamp'],axis=1)
    mov_rating_mean = mov_rating.groupby('Title').mean().sort_values(by=['count','Rating'], ascending=False)
    return mov_rating_mean.head(5)

#testing: only show results for the first two genres
for i in genres_list[:2]:
    
    recc_genre = recc_genre_by_popularity(movies,ratings,i).drop(['Rating'],axis=1)
    print('reccomendations for the genre:',i,'\n')
    print(recc_genre.head())
    print('\n')

reccomendations for the genre: Fantasy 

                                              count  avg_rating
Title                                                          
Star Wars: Episode IV - A New Hope (1977)      2991    4.453694
E.T. the Extra-Terrestrial (1982)              2269    3.965183
Big (1988)                                     1491    3.855801
Willy Wonka and the Chocolate Factory (1971)   1313    3.861386
Heavenly Creatures (1994)                       477    3.865828


reccomendations for the genre: War 

                                                    count  avg_rating
Title                                                                
Star Wars: Episode V - The Empire Strikes Back ...   2990    4.292977
Saving Private Ryan (1998)                           2653    4.337354
Schindler's List (1993)                              2304    4.510417
Casablanca (1942)                                    1669    4.412822
Dr. Strangelove or: How I Learned to Stop Worry...  

# Collaborative filtering

SVD is a well-extablished matrix factorization based algorithm. Specifically, SVD decompose a complex m×n matrix into three matrixes, M = U∑V*.  However, SVD cannot be applied directly to matrix with many missing values. An common approach to deal with missing values is imputation. For example, we can fill in the missing ratings with the mean value of user rating. However, imputation increase the amount of data and distort the data due to inaccurate imputation.  Koren et al proposed an algorithm which directly models available rating data, 
The objective function is defined as follows:

$\sum_{}^{} (r_{ui} -\left ( \mu  + b_{u} + b_{i} \right ))^{2} +\lambda \left ( b_{u}^{2} +b_{i}^{2} \right )$

In [10]:
# !pip install scikit-surprise

In [11]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
data = Dataset.load_from_df(ratings[['UserID', 'MovieID', 'Rating']], reader)

In [12]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10,verbose=False)

{'test_rmse': array([0.86788407, 0.8641075 , 0.86743876, 0.86664634, 0.86639098,
        0.86659294, 0.86703767, 0.86504927, 0.86191395, 0.86733213]),
 'test_mae': array([0.68017162, 0.67761849, 0.67934242, 0.67942749, 0.67943753,
        0.6794477 , 0.68011645, 0.67931991, 0.67635695, 0.68078512]),
 'fit_time': (60.49817442893982,
  60.045353174209595,
  63.91104054450989,
  60.55002951622009,
  61.286060094833374,
  59.996570110321045,
  61.49649691581726,
  60.32887506484985,
  59.557671546936035,
  60.098212003707886),
 'test_time': (1.0352246761322021,
  1.1658813953399658,
  1.38529634475708,
  1.037259817123413,
  1.035243272781372,
  1.0481979846954346,
  1.0452032089233398,
  1.08012056350708,
  0.972400426864624,
  1.0841007232666016)}

 ## User Based Collabrative Filter Cross Validation
 

In [13]:
#Cross Validation
from surprise import KNNBasic
sim_options = {'name': 'cosine',
               'user_based': True 
               }
algo_ub = KNNBasic(sim_options=sim_options)
cross_validate(algo_ub, data, measures=['RMSE', 'MAE'], cv=10,verbose=False)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.9699062 , 0.97509635, 0.97570597, 0.97940029, 0.97223726,
        0.97675494, 0.97701525, 0.97160571, 0.98008486, 0.97577582]),
 'test_mae': array([0.76577174, 0.76917915, 0.77102   , 0.77023031, 0.7664974 ,
        0.77030906, 0.77093525, 0.76515559, 0.77249257, 0.77024554]),
 'fit_time': (135.3540518283844,
  127.99304485321045,
  125.97701382637024,
  143.4173150062561,
  176.80064153671265,
  181.56185817718506,
  153.73285746574402,
  163.16774678230286,
  182.0464186668396,
  179.94403386116028),
 'test_time': (80.80587863922119,
  79.03554511070251,
  75.7384340763092,
  105.66220211982727,
  109.06240797042847,
  107.11631512641907,
  87.67414927482605,
  116.5457649230957,
  111.34472417831421,
  111.87123107910156)}

## User Based Collabrative Filter for new users

To speed up the UBCF algorithm in the Dash APP, users data are filtered before calculating cosine_similarity. The general procedures for UBCF are as follows:
1.	Get new user’s rating data (N by 2 list of lists) in the format: [[moviesID1,rating1], [moviesID2,rating2]...[moviesID_N,ratingN]]
2.	Filter ratings data, find moviesIDs in the database that are rated by both the new user and existing users. 
3.	Transform filtered rating data into pivot table, fill in empty spaces with 0.
4.	Calculate cosine similarity between new user and old users, find k most similar users (k was set to 3).
5.	Calculate the average ratings of the movies rated by the most similar users from step 4. Filter the movies, only the movies that received more than 50% of ratings than other movies are considered for the final list. 
6.	Sort movies’ list by ratings, return the movies with highest ratings.


In [16]:

#create a new user for testing
# user_rating_list = [[i+1,i+2] for i in range(3)]
user_rating_list = [[1, 5], [2, 5], [3, 4],[4, 1]]# [MovieID, rating]
print('New user:',user_rating_list)
#---------------------------------------------------
new_user_ratings = [i[1] for i in user_rating_list] 
new_user_movies= [i[0] for i in user_rating_list] 
new_user_ratings_df =pd.DataFrame(np.array(new_user_ratings).reshape(1,-1),columns=new_user_movies)

# find users in the database that also rated the movies rated by new user
filtered_ratings = ratings[ratings['MovieID'].isin(new_user_movies)].drop('Timestamp',axis=1)
filtered_rating_matrix = filtered_ratings.pivot_table(index='UserID', columns='MovieID', values='Rating').fillna(0)
# print(filtered_rating_matrix[:5])


def similar_users(new_user_ratings, other_users, k=3):
#     https://towardsdatascience.com/build-a-user-based-collaborative-filtering-recommendation-engine-for-anime-92d35921f304
    new_user_ratings =new_user_ratings.copy()
    # calc cosine similarity between user and each other user
    similarities = cosine_similarity(new_user_ratings,other_users)[0].tolist()  
    
    # create list of indices of these users
    indices = other_users.index.tolist()  
    # create key/values pairs of user index and their similarity
    index_similarity = dict(zip(indices, similarities))
    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1),reverse =True)
    
    # grab k users off the top
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    return users

similar_user_list= similar_users(new_user_ratings_df,filtered_rating_matrix ,5)
print('found similar users:',similar_user_list)

New user: [[1, 5], [2, 5], [3, 4], [4, 0]]
found similar users: [223, 731, 308, 310, 1264]


In [17]:
def UBCF_recc_movies(ratings,similar_user_list,k=6):
    
    similar_ratings =ratings[ratings['UserID'].isin(similar_user_list)].drop('Timestamp',axis=1) 
    similar_ratings['count']=similar_ratings['UserID'].groupby(similar_ratings['MovieID']).transform('count')
    #only movies that received more number of ratings than 80% of all the movies in the genre are considered
    cutoff = similar_ratings['count'].quantile(0.8)
    similar_ratings = similar_ratings[similar_ratings['count']>=cutoff]
    movie_avg_ratings = similar_ratings.groupby('MovieID').mean().sort_values(by=['Rating'], ascending=False).reset_index()
#     print(movie_avg_ratings)
    movies_list = list(movie_avg_ratings['MovieID'])
    return str(movies_list[:6])


movies_list = UBCF_recc_movies(ratings,similar_user_list,6)
print('Reccomended MovieIDs:',movies_list)

Reccomended MovieIDs: [527, 1247, 858, 593, 1961, 2997]


# References
1.	Kaggle Post: Movies Recommender System. https://www.kaggle.com/rounakbanik/movie-recommender-systems
2.	Recommender Systems — User-Based and Item-Based Collaborative Filtering. https://medium.com/@cfpinela/recommender-systems-user-based-and-item-based-collaborative-filtering-5d5f375a127f
3.	Kaggle Post: https://www.kaggle.com/rangarajansaranathan/collaborative-filtering-based-recommender-system
4.	Yehuda Koren, Collaborative Filtering with Temporal Dynamics. doi:10.1145/1721654.1721677
5.	Surprise documentation, https://surprise.readthedocs.io/en/stable/index.html?highlight=reference
